### Consider the expression syntax tree discussed here: https://ruslanspivak.com/lsbasi-part7/ . Develop a python program that generates a set of random expressions based on the syntax tree definition. (e.g., this may be useful to generate synthetic expression dataset)




### **GRAMMAR**

###       expr: term ((PLUS | MINUS) term)*
  ###       term: factor ((MUL | DIV) factor)*
  ###       factor: Integer | Lparen expr Rparen

In [ ]:
import random


#  LEXER


INTEGER, PLUS, MINUS, MUL, DIV, LPAREN, RPAREN, EOF = (
    'INTEGER', 'PLUS', 'MINUS', 'MUL', 'DIV', 'LPAREN', 'RPAREN', 'EOF'
)

class Token():
    def __init__(self, type, value):
        self.type = type
        self.value = value

    def __str__(self):
        return 'Token({type}, {value})'.format(
            type=self.type,
            value=repr(self.value)
        )

    def __repr__(self):
        return self.__str__()

class Lexer():
    def __init__(self, expression):
        self.expression = expression
        self.pos = 0
        self.current_char = self.expression[self.pos]


    def advance(self):
        self.pos += 1
        if self.pos > len(self.expression) - 1:
            self.current_char = None
        else:
            self.current_char = self.expression[self.pos]

    def skip_whitespace(self):
        while self.current_char is not None and self.current_char.isspace():
            self.advance()

    def integer(self):
        result = ''
        while self.current_char is not None and self.current_char.isdigit():
            result += self.current_char
            self.advance()
        return int(result)

    def get_next_token(self):
        while self.current_char is not None:
            if self.current_char.isspace():
                self.skip_whitespace()
                continue
            if self.current_char.isdigit():
                return Token(INTEGER, self.integer())
            if self.current_char == '+':
                self.advance()
                return Token(PLUS, '+')
            if self.current_char == '-':
                self.advance()
                return Token(MINUS, '-')
            if self.current_char == '*':
                self.advance()
                return Token(MUL, '*')
            if self.current_char == '/':
                self.advance()
                return Token(DIV, '/')
            if self.current_char == '(':
                self.advance()
                return Token(LPAREN, '(')
            if self.current_char == ')':
                self.advance()
                return Token(RPAREN, ')')
        return Token(EOF, None)


#  PARSER


class AST():
    pass

class BinOp(AST):
    def __init__(self, left, op, right):
        self.left = left
        self.token = self.op = op
        self.right = right

class Num(AST):
    def __init__(self, token):
        self.token = token
        self.value = token.value

class Parser():
    def __init__(self, lexer):
        self.lexer = lexer
        self.current_token = self.lexer.get_next_token()

    def error(self):
        raise Exception('Invalid syntax')

    def eat(self, token_type):
        if self.current_token.type == token_type:
            self.current_token = self.lexer.get_next_token()
        else:
            self.error()

    def factor(self):
        token = self.current_token
        if token.type == INTEGER:
            self.eat(INTEGER)
            return Num(token)
        elif token.type == LPAREN:
            self.eat(LPAREN)
            node = self.expr()
            self.eat(RPAREN)
            return node

    def term(self):
        node = self.factor()
        while self.current_token.type in (MUL, DIV):
            token = self.current_token
            if token.type == MUL:
                self.eat(MUL)
            elif token.type == DIV:
                self.eat(DIV)
            node = BinOp(left=node, op=token, right=self.factor())
        return node

    def expr(self):
        node = self.term()
        while self.current_token.type in (PLUS, MINUS):
            token = self.current_token
            if token.type == PLUS:
                self.eat(PLUS)
            elif token.type == MINUS:
                self.eat(MINUS)
            node = BinOp(left=node, op=token, right=self.term())
        return node

    def parse(self):
        return self.expr()


#  INTERPRETER


class NodeVisitor():
    def visit(self, node):
        method_name = 'visit_' + type(node).__name__
        visitor = getattr(self, method_name, self.generic_visit)
        return visitor(node)

    def generic_visit(self, node):
        raise Exception('No visit_{} method'.format(type(node).__qualname__))

class Interpreter(NodeVisitor):
    def __init__(self, parser):
        self.parser = parser

    def visit_BinOp(self, node):
        if node.op.type == PLUS:
            return self.visit(node.left) + self.visit(node.right)
        elif node.op.type == MINUS:
            return self.visit(node.left) - self.visit(node.right)
        elif node.op.type == MUL:
            return self.visit(node.left) * self.visit(node.right)
        elif node.op.type == DIV:
          try:
            return self.visit(node.left) / self.visit(node.right)
          except ZeroDivisionError:
            print("Error: Division by zero")


    def visit_Num(self, node):
        return node.value

    def interpret(self):
        tree = self.parser.parse()
        return self.visit(tree)


#  RANDOM EXPRESSION GENERATOR

# Class to represent an expression in parentheses
class ParenExpr:
    def __init__(self, expr):
        self.expr = expr

# Function to generate a random factor (number or expression in parentheses)
def generate_random_factor(depth):
    # Base case: if depth exceeds 3 or randomly chosen to stop, return a number node
    if depth > 3 or random.choice([True, False]):
        return Num(Token(INTEGER, random.randint(1, 10)))
    else:
        # Recursive case: generate a new expression with increased depth
        return ParenExpr(generate_random_expr(depth + 1))

# Function to generate a random term (factor possibly followed by * or / operations)
def generate_random_term(depth):
    # Start with a factor node
    node = generate_random_factor(depth)
    # Randomly decide whether to add multiplication or division
    while random.choice([True, False]):
        token_type, token_symbol = random.choice([(MUL, '*'), (DIV, '/')])
        token = Token(token_type, token_symbol)
        # Create a binary operation node with the current node and a new factor
        node = BinOp(left=node, op=token, right=generate_random_factor(depth))
    return node

# Function to generate a random expression (term possibly followed by + or - operations)
def generate_random_expr(depth):
    # Start with a term node
    node = generate_random_term(depth)
    # Randomly decide whether to add addition or subtraction
    while random.choice([True, False]):
        token_type, token_symbol = random.choice([(PLUS, '+'), (MINUS, '-')])
        token = Token(token_type, token_symbol)
        # Create a binary operation node with the current node and a new term
        node = BinOp(left=node, op=token, right=generate_random_term(depth))
    return node

# Function to convert the AST to a string representation
def ast_to_string(node):
    # If the node is a number, return its value as a string
    if isinstance(node, Num):
        return str(node.value)
    # If the node is a binary operation
    elif isinstance(node, BinOp):
        return f'{ast_to_string(node.left)} {node.op.value} {ast_to_string(node.right)}'
    # If the node is a parenthesized expression, return it with parentheses
    elif isinstance(node, ParenExpr):
        return f'({ast_to_string(node.expr)})'


def main():
    num_expressions = 5
    expressions = [generate_random_expr(2) for _ in range(num_expressions)]

    print("Generated Expressions:")
    for expr_ast in expressions:
        print(ast_to_string(expr_ast))
    print()

    for expr_ast in expressions:
        expression = ast_to_string(expr_ast)
        print(f'Expression: {expression}')
        lexer = Lexer(expression)

        tokens = []
        while True:
            token = lexer.get_next_token()
            tokens.append(token)
            if token.type == EOF:
                break

        print("Tokens:")
        for token in tokens:
            print(token)

        parser = Parser(Lexer(expression))
        interpreter = Interpreter(parser)
        result = interpreter.interpret()
        print(f'Result: {result}\n')

if __name__ == '__main__':
    main()


Generated Expressions:
((10) / (1) - (6 * 10 - 9 - 8 / 2 + 6 * 2)) * ((10 * 6 + 5 * 4 / 1 * 10 / 6 / 5 - 2) * (3 - 4) * 8 + 2 / 1) * 1 * ((7 / 1 * 3 - 6 * 2 / 8) / 5 / (6 * 4 + 6) / (3 / 8 * 4 * 8) + 4 * 1 * 6 * 10 * 6) - (10 * (8 / 1 - 9 * 7 / 9) + (1 / 7 / 8) - 8)
(6 / 1 / 3 - (4 * 8 * 6 - 6) / 5 / (3))
6
6 - ((1 * 4 / 5 - 9 - 7 * 10) / (7 - 8 - 9 / 6 * 10 / 8 + 8 / 8 * 3 / 6) / (3) - 10)
((9 * 2 + 4))

Expression: ((10) / (1) - (6 * 10 - 9 - 8 / 2 + 6 * 2)) * ((10 * 6 + 5 * 4 / 1 * 10 / 6 / 5 - 2) * (3 - 4) * 8 + 2 / 1) * 1 * ((7 / 1 * 3 - 6 * 2 / 8) / 5 / (6 * 4 + 6) / (3 / 8 * 4 * 8) + 4 * 1 * 6 * 10 * 6) - (10 * (8 / 1 - 9 * 7 / 9) + (1 / 7 / 8) - 8)
Tokens:
Token(LPAREN, '(')
Token(LPAREN, '(')
Token(INTEGER, 10)
Token(RPAREN, ')')
Token(DIV, '/')
Token(LPAREN, '(')
Token(INTEGER, 1)
Token(RPAREN, ')')
Token(MINUS, '-')
Token(LPAREN, '(')
Token(INTEGER, 6)
Token(MUL, '*')
Token(INTEGER, 10)
Token(MINUS, '-')
Token(INTEGER, 9)
Token(MINUS, '-')
Token(INTEGER, 8)
Token(DIV, '/')
T